# Exploring the FORCE 2020 Well Log Challenge - Part 2
## Map plots 

**Brendon Hall, Enthought**

bhall@enthought.com

Welcome back!  In the [first notebook](https://github.com/brendonhall/FORCE-2020-Lithology/blob/master/notebooks/01-Log-Plot-MPL.ipynb), I showed how to use `matplotlib` to display any well curves for any well in the data supplied for the [2020 FORCE Machine Learning Contest](https://xeek.ai/challenges/force-well-logs/overview). In this notebook, I'm going to use `plotly` to display the well locations on a map.  This will help build an intuition for how the wells are related spatially. Wells that are closer together might have properties that are more correlated with each other, and this could be a useful fact to exploit when building machine learning models to predict log curves and lithofacies. 

Please get in touch if you have any questions.  You can also join in the conversation on [Software Underground's slack](https://softwareunderground.org/slack) in the **#force_2020_ml_contest** channel.

Feel free to use this code, hack it, adapt it for your own needs.

The well log data is licensed as [Norwegian License for Open Government Data (NLOD) 2.0](https://data.norge.no/nlod/en/2.0/).
The well log labels that are included are provided by FORCE 2020 Machine Learning Contest under [CC-BY-4.0](https://creativecommons.org/licenses/by/4.0/).

In [1]:
import os.path

import numpy as np
import pandas as pd

import plotly.express as px

pd.options.display.max_rows = 8

Let's load the training data.  If you have already downloaded it, set `local_train_csv` to the location of the training data on your computer.  If not, you can run the cell below and it will download the data from AWS S3, and save it to the location specified by `local_train_csv`.

In [2]:
# change this to the location of the training data on your disk if
# you have already downloaded it
local_train_csv = 'train.csv'

if not os.path.isfile(local_train_csv):
    # load from s3
    s3_train_csv = 's3://zarr-depot/wells/FORCE: Machine Predicted Lithology/train.csv'
    train_df = pd.read_csv(s3_train_csv, sep=';')
    train_df.to_csv(local_train_csv, index=False, sep=';')
    
else:
    # load from disk
    train_df = pd.read_csv(local_train_csv)
    
train_well_names = train_df['WELL'].unique()

We're going to look at the test wells as well in the tutorial and see where these wells are located with respect to the training data.  Same as above, this cell will download the test data if you haven't already done so.

In [3]:
# change this to the location of the test data on your disk if
# you have already downloaded it
local_test_csv = 'test.csv'

if not os.path.isfile(local_test_csv):
    # load from s3
    s3_test_csv = 's3://zarr-depot/wells/FORCE: Machine Predicted Lithology/test.csv'
    test_df = pd.read_csv(s3_test_csv, sep=';')
    test_df.to_csv(local_test_csv, index=False, sep=';')
    
else:
    # load from disk
    test_df = pd.read_csv(local_test_csv, sep=';')

test_well_names = test_df['WELL'].unique()

For this tutorial we only need the unique well names from both datasets.  Let's combine the two name arrays and create a dataframe with just this column.  We'll need it to merge with the well meta data later.

In [57]:
well_names = np.concatenate((train_well_names, test_well_names))
# need this array in a DataFrame for a merge operation below
well_names_df = pd.DataFrame({'WELL':well_names})

well_names_df

,WELL
0,15/9-13
1,15/9-15
2,15/9-17
3,16/1-2
...,...
104,34/3-3 A
105,34/6-1 S
106,35/6-2 S
107,35/9-8


There are a total of 108 wells in the combined dataset (98 training, and 10 test wells)

Alright, now let's take a look at some well 'meta' data from the [NPD Factpages summary](https://factpages.npd.no/en/wellbore/tableview/exploration/all) page.  Click `Export CSV` just above the first row of the table to save the data to your disk.

This data contains a wealth of information about the wells themselves like the operator, type, field, formations and of course location.  Both UTM coordinates are given, as well as latitude and longitude.  We'll use a small subset of the data to display with the wells in our map plot.  Check out the columns on the NPD website and consider what they mean.  There might be some useful information we can use for feature engineering.

In [36]:
# location of the meta data csv on your machine 
well_meta_csv = '../data/wellbore_exploration_all.csv'
well_meta_df = pd.read_csv(well_meta_csv)

if not os.path.isfile(well_meta_csv):
    # load from s3
    s3_meta_csv = 's3://zarr-depot/wells/FORCE: Machine Predicted Lithology/wellbore_exploration_all.csv'
    well_meta_df = pd.read_csv(s3_test_csv)
    well_meta_df.to_csv(well_meta_csv, index=False)
    
else:
    # load from disk
    well_meta_df = pd.read_csv(well_meta_csv)

# rename the columns so they are more readable
well_meta_df.rename(columns={'wlbWellboreName': 'WELL',
                             'wlbWell': 'WELL_HEAD',
                            'wlbNsDecDeg': 'lat',
                            'wlbEwDesDeg': 'lon',
                            'wlbDrillingOperator': 'Drilling Operator',
                            'wlbPurposePlanned': 'Purpose',
                            'wlbCompletionYear': 'Completion Year',
                            'wlbFormationAtTd': 'Formation'}, inplace=True)

# get df of WELL_HEAD and the lat long
well_locations_df = well_meta_df[['WELL_HEAD', 'lat', 'lon']].drop_duplicates(subset=['WELL_HEAD'])

# we only need a few of the columns for the map plot
well_meta_df = well_meta_df[['WELL','Drilling Operator',
                            'Purpose','Completion Year', 'Formation']].copy()

well_locations_df

,WELL_HEAD,lat,lon
0,1/2-1,56.887519,2.476583
1,1/2-2,56.992222,2.496572
2,1/3-1,56.855833,2.851389
3,1/3-2,56.936111,2.750000
...,...,...,...
1948,7325/1-1,73.913528,25.116714
1949,7325/4-1,73.649319,25.178261
1950,7335/3-1,73.997183,35.837147
1951,7435/12-1,74.071725,35.808628


As you can see there are quite a few wells (almost 2000) in the meta data file.  We need to compare the wells we have in our dataset to these wells to extract the corresponding meta data.  The wells are named using a scheme outlined in the [NPD Guidelines for Designation of Wells and Wellbores](https://www.npd.no/globalassets/1-npd/regelverk/tematiske-veiledninger/eng/guidelines-for-designation-of-wells-and-wellbores.pdf).  Wells are identified using the following form:

[Quadrant Number]/[Block Number]-[Well Bore ID] [Sidetracks, etc]

For example, there are a couple of wells in the dataset named: 

`34/5-1 A`, and  `34/5-1 S`

These wells have the same well head and represent different sidetracks (and similar).  The meta file doesn't contain info for every sidetrack that is represented in our training data.  So let's extract just the well head prefix from the well names, and use that to get the locations for the wells.

In [53]:
def base_well_name(row):
    
    well_name = row['WELL']
    
    return well_name.split()[0]

# apply the function to extract the WELL_HEAD base name from the well
well_names_df['WELL_HEAD'] = well_names_df.apply(lambda row: base_well_name(row), axis=1)

# merge with location data to get lat/lon
locations_df = well_names_df.merge(well_locations_df, how='inner', on='WELL_HEAD')
# merge with the meta data to get other data
locations_df = locations_df.merge(well_meta_df, how='left', on='WELL')
locations_df 

,WELL,WELL_HEAD,lat,lon,Drilling Operator,Purpose,Completion Year,Formation
0,15/9-13,15/9-13,58.373878,1.934128,Den norske stats oljeselskap a.s,APPRAISAL,1982.0,ZECHSTEIN GP
1,15/9-15,15/9-15,58.302069,1.922131,Den norske stats oljeselskap a.s,WILDCAT,1982.0,SKAGERRAK FM
2,15/9-17,15/9-17,58.445608,1.948217,Den norske stats oljeselskap a.s,WILDCAT,1983.0,SMITH BANK FM
3,16/1-2,16/1-2,58.935894,2.222239,Esso Exploration and Production Norway A/S,APPRAISAL,1976.0,BASEMENT
...,...,...,...,...,...,...,...,...
104,34/3-3 A,34/3-3,61.795136,2.717883,BG Norge AS,APPRAISAL,2012.0,BURTON FM
105,34/6-1 S,34/6-1,61.582317,2.685472,Norske Conoco A/S,WILDCAT,2002.0,LUNDE FM
106,35/6-2 S,35/6-2,61.533606,3.911311,StatoilHydro Petroleum AS,WILDCAT,2009.0,NO FORMAL NAME
107,35/9-8,35/9-8,61.285269,3.675594,Wintershall Norge AS,APPRAISAL,2013.0,RANNOCH FM


We have data for all 108 wells in the dataset.

Now let's add a column to the well location dataframe that indicates if the well is a training well, or if it comes from the test dataset.

In [58]:
locations_df.loc[locations_df['WELL'].isin(train_well_names), 'Dataset'] = 'Train'
locations_df.loc[locations_df['WELL'].isin(test_well_names), 'Dataset'] = 'Test'

# save the location and meta data for future use.
locations_df.to_csv('../data/force_2020_meta.csv', index=False)

Finally we can use this location data to plot the well locations on a map.  I'm going to use Plotly's [`scatter_mapbox`](https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter_mapbox.html) to do this.  There are plenty of Python based mapping options out there, but in the next notebook I'm going to build an interactive dashboard using Plotly's Dash framework, and we can just drop this plot in there.

For this plot, we'll view the well locations using their latitude and longitude coordinates using Plotly's [mapbox](https://www.mapbox.com/) interface.  This uses the `open-street-map` style, so we don't need an API key.  There are a number of [cool styles](https://plotly.com/python/mapbox-layers/) mapbox offers to change the look of your plot.  Some of them require you to [sign up](https://docs.mapbox.com/help/how-mapbox-works/access-tokens/) for an API key.

The wells are indicated by colored dots. Wells from the training dataset are blue, wells in the test set are red. The meta data associated with each well is displayed when you hover your mouse over the well location.  This is easy to configure using the `hover_data` parameter in the `scatter_mapbox` function below.

In [59]:
fig = px.scatter_mapbox(locations_df, lat="lat", lon="lon",
                        color='Dataset', 
                        zoom=5, height=600,
                        hover_data={'WELL': True,
                                    'lat': False,
                                    'lon': False,
                                    'Dataset': False,
                                    'Drilling Operator': True,
                                    'Purpose': True,
                                    'Completion Year': True,
                                    'Formation': True}
                        )
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":200,"t":20,"l":200,"b":0})
fig.show()

It seems like there could be (at least) three clusters of points in the contest data.  Two northern clusters and an elongated one to the south.  Each cluster has at least a couple of test wells.  Perhaps a clustering algorithm (like kmeans) could be used to assign a spatial cluster ID might be a useful feature for machine learning?  Something to test in the near future.

In the next notebook we'll build on this map plot, and add some interactivity using Plotly's Dash framework.  I'll show how to build a tool to select and visualize groups of wells not only based on location, but also what curves each well possesses.

This notebook is open source content. Text is CC-BY-4.0, code is [Apache 2.0](https://www.apache.org/licenses/LICENSE-2.0).

### References

Bormann P., Aursand P., Dilib F., Dischington P., Manral S. (2020) 2020 FORCE Machine Learning Contest. https://github.com/bolgebrygg/Force-2020-Machine-Learning-competition